In [79]:
import pandas as pd
import datacompy
import gc
import os
import re
# Ouali benchmark file only includes auto aligned data
benchmark_file = 'input/rapport_20240806144726_alignement-défini_TEST ATC-_IdRef_extraction_patrinum_min2align.tsv'
abes_files = ['input/abes-alignements-surs.csv']
abes_experimental_files = ['input/abes-alignements-experimentaux-20220602.csv','input/abes-alignements-experimentaux-2efichier-methode_a-20220725.csv']
abes_noalign_file = 'input/abes-nonalign-only.csv'

# If you'd like to compare alignments that are common between Ouali and ABES with a set of manual/external alignments
# speficy it here (or leave empty to bypass this step).
ouali_to_validate_file = 'input/rapport_20240809012534_alignement-défini_TEST ATC-_IdRef_extraction_patrinum_extmanuels.tsv'

# Optionally, provide a file containing already computed common alignements (if provided, other steps are bypassed)
common_alignments_file = 'output/communs-ABES-benchmark2024-min2align.csv'

# Output files will include this prefix
set_name = 'benchmark2024-min2align'

output_folder = 'output'

# Export as CSV or Excel?
output_type = 'xlsx'

# Main names for various export files
common_file = 'communs'
common_noalign_file = 'communs-nonalignements'
divergences_file = 'divergences'
div_noalign_file = 'divergences-nonalignements'
missed_file = 'manquants'

# Check that given filenames do indeed exist
all_files = []
for variable in [benchmark_file, abes_files, abes_experimental_files, abes_noalign_file, ouali_to_validate_file, common_alignments_file]:
    if isinstance(variable, list):
        all_files.extend(variable)
    else:
        all_files.append(variable)

for file in all_files:
    if file and not os.path.isfile(file):
        print("ATTENTION le fichier suivant n'existe pas: ",file)

### Préparation des données Ouali

Commençons par charger les données exportées depuis Ouali en les chargeant dans des dataframes.

Pour tous les chargements de données, on s'assure que Python traite toutes les valeurs comme texte, sinon il va convertir certaines colonnes contenant des identifiants en nombres, ce qui peut poser problème par la suite si on les compare aux mêmes données chargées comme texte ou si on essaie de faire une opération join sur ces colonnes. Pour cela, on spécifie le paramètre `dtype`.

In [60]:
ouali_data = pd.read_csv(benchmark_file, sep='\t', dtype = str)
#ouali_undefined_data = pd.read_csv(benchmark_undefined_file, sep='\t', dtype = str)
# Create origin and target columns that are named the same as in the RERO files, for ease of merging
ouali_data['source'] = ouali_data['id source']
#ouali_undefined_data['source'] = ouali_undefined_data['id source']
ouali_data['cible'] = ouali_data['id cible']
# .copy() is necessary to avoid SettingWithCopyWarning when merging
ouali_align = ouali_data.query('`nombre de candidats` != "0" & `décision d\'alignement` == "auto"').copy()
ouali_no_align = ouali_data.query('`nombre de candidats` == "0" & `décision d\'alignement` == "auto"').copy()
    
print('Alignements Ouali chargés: ' + str(len(ouali_align)))
print('Non-alignements Ouali chargés: ' + str(len(ouali_no_align)))
#print('Alignements à arbitrer (pas de décision): ' + str(len(ouali_undefined_data)))

Alignements Ouali chargés: 293929
Non-alignements Ouali chargés: 53374


### Préparation des données ABES

Les données ABES sont chargées dans des dataframes. Pour les alignements expérimentaux, ceux-ci sont répétés lorsqu'il y a plusieurs notices bib liées. On déduplique donc ce fichier pour ne conserver qu'une ligne par alignement calculé.

In [61]:
if not common_alignments_file:
    abes_align = pd.concat((pd.read_csv(f, dtype = str) for f in abes_files))
    abes_experimental_align = pd.concat((pd.read_csv(f, dtype = str) for f in abes_experimental_files ))
    abes_noalign = pd.read_csv(abes_noalign_file, dtype = str)
    
    abes_experimental_align.drop_duplicates(subset=['auth_id','IdRef_CANDIDAT'], inplace=True)
    print("Nombre de concordances validées dans le fichier ABES: ",len(abes_align))
    print("Nombre de concordances validées dans le fichier expérimental ABES: ",len(abes_experimental_align))
    print("Nombre de non-alignements validés dans le fichier ABES: ",len(abes_noalign))

### Comparaison des données Ouali avec l'ABES

Dans les données ABES, l'identifiant RNV est au milieu de la chaîne de caractères présents dans la colonne `ID_EC`. Il faut donc extraire ces chiffres et les placer dans une nouvelle colonne `source` pour pouvoir les comparer à Ouali. On renomme également la colonne `IdRef` en `cible` pour faciliter la comparaison. On fait la même chose pour le fichier des non-alignements.

Les fichiers expérimentaux de l'ABES ont un autre format qu'il faut adapter. Si les fichiers expérimentaux apportent des doublons par rapport aux alignements ABES "sûrs", seuls ces derniers sont conservés.

In [62]:
if not common_alignments_file:
    abes_align['source']=abes_align['ID_EC'].str.extract(r'(\d{18})')
    abes_align['cible']=abes_align['IdRef']
    abes_noalign['source']=abes_noalign['ID_EC'].str.extract(r'(\d{18})')
    
    abes_experimental_align['source']=abes_experimental_align['auth_id']
    abes_experimental_align['cible']=abes_experimental_align['IdRef_CANDIDAT']
    abes_experimental_align['NOM']=abes_experimental_align['auth_heading']
    abes_experimental_align['NOMCANDIDAT']=abes_experimental_align['Appellation_IdRef']
    
    all_abes = pd.concat([abes_align, abes_experimental_align])
    all_abes.drop_duplicates(subset=['source'], inplace=True, keep='first')
    
    print("Nombre de concordances ABES (valides + expérimentales): ",len(all_abes))
    display(all_abes)

In [63]:
if not common_alignments_file:
    common_source_abes = pd.merge(all_abes, ouali_align, how="inner", on='source', suffixes=("_abes", "_ouali"), copy=True)
    common_source_noalign_abes = pd.merge(abes_noalign, ouali_no_align, how="inner", on='source', suffixes=("_abes", "_ouali"), copy=True)
    print("Alignements Ouali présents dans fichier ABES: ", len(common_source_abes))

In [64]:
if not common_alignments_file:
    outfile = output_folder + '/' + common_file + '-ABES-' + set_name

    common_target_abes = pd.merge(all_abes, ouali_align, how="inner", on=['source', 'cible'], suffixes=("_abes", "_ouali"), copy=True)
    
    if output_type == 'xlsx':
        outfile = outfile + '.xlsx'
        common_target_abes.to_excel(outfile,columns=['source','cible','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT'],index=False)
    else:
        outfile = outfile + '.csv'
        common_target_abes.to_csv(outfile,columns=['source','cible','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT'],encoding="UTF-8",index=False)
        
    print("Alignements Ouali validés par l'ABES: ", len(common_target_abes))
    print("Pourcentage validé: ", round(len(common_target_abes)/len(common_source_abes)*100,1), "%")
    print("Exportés dans ", outfile)
else:
    common_target_abes = pd.read_csv(common_alignments_file, dtype = str)
    print("Fichier d'alignements communs externes chargés")
    print("Alignements Ouali validés par l'ABES: ", len(common_target_abes))


Fichier d'alignements communs externes chargés
Alignements Ouali validés par l'ABES:  212146


In [65]:
if not common_alignments_file:
    outfile = output_folder + '/' + common_noalign_file + '-ABES-' + set_name
    if output_type == 'xlsx':
        outfile = outfile + '.xlsx'
        common_source_noalign_abes.to_excel(outfile,columns=['source'],index=False)
    else:
        outfile = outfile + '.csv'
        common_source_noalign_abes.to_csv(outfile,columns=['source'],encoding="UTF-8",index=False)
        
    print("Non-alignements Ouali: ", len(ouali_no_align))
    print("Non-alignements Ouali communs avec l'ABES: ", len(common_source_noalign_abes))
    print("Pourcentage validé: ", round(len(common_source_noalign_abes)/len(ouali_no_align)*100,1), "%")
    print("Exportés dans ", outfile)

#### Différences entre Ouali et l'ABES

In [66]:
if not common_alignments_file:
    outfile = output_folder + '/' + divergences_file + '-ABES-' + set_name + '.csv'
    # Alignements pour lesquels Ouali et l'ABES ont pris une décision différente (cible différente pour une même source)
    divergences_abes = common_source_abes.loc[~(common_source_abes['cible_abes'] == common_source_abes['cible_ouali'])]
    display(divergences_abes[['source','cible_ouali','cible_abes','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT']])
    
    # Notices qu'Ouali a alignées, mais l'ABES a pris une décision de non-alignement
    divergences_NOT_abes = pd.merge(abes_noalign, ouali_align, how="inner", on=['source'], suffixes=("_abes", "_ouali"), copy=True)
    divergences_NOT_abes['cible_ouali'] = divergences_NOT_abes['cible']
    divergences_NOT_abes.insert(0,'cible_abes', "NaN")
    display(divergences_NOT_abes[['source','cible_ouali','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT']])
    
    divergences_abes.to_csv(outfile,columns=['source','cible_ouali','cible_abes','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT'],encoding="UTF-8",index=False)
    # On ajoute à ce même fichier les cas où l'ABES a pris une décision de non-alignement (mode 'a' - append)
    divergences_NOT_abes.to_csv(outfile,columns=['source','cible_ouali','cible_abes','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT'],encoding="UTF-8",index=False, mode='a')
    
    
    print("Alignements Ouali qui diffèrent dans l'ABES: ", len(divergences_abes))
    print("Pourcentage avec alignement différent: ", round(len(divergences_abes)/len(common_source_abes)*100,1), "%")
    print("Alignements Ouali qui sont non-alignés par l'ABES: ", len(divergences_NOT_abes))
    print("Pourcentage avec décision d'alignement différente: ", round(len(divergences_NOT_abes)/len(ouali_align)*100,1), "%")
    print("Exportés dans ", outfile)

In [67]:
if not common_alignments_file:
    outfile = output_folder + '/' + div_noalign_file + '-ABES-' + set_name + '.csv'

    # Non-alignements Ouali pour lesquels l'ABES a trouvé un alignement
    div_nonalign_abes = pd.merge(abes_align, ouali_no_align, how="inner", on='source', suffixes=("_abes", "_ouali"), copy=True)
    display(div_nonalign_abes[['source','cible_ouali','cible_abes','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT']])
    
    div_nonalign_abes.to_csv(outfile,columns=['source','cible_ouali','cible_abes','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT'],encoding="UTF-8",index=False)
    
    print("Non-alignements Ouali erronés: ", len(div_nonalign_abes))
    print("Pourcentage de divergence: ", round(len(div_nonalign_abes)/len(ouali_no_align)*100,1), "%")
    
    print("Exportés dans ", outfile)


#### Validation d'alignements à l'aide des alignements Ouali automatiques
Si un fichier à valider a été fourni, celui-ci va maintenant être comparé aux alignements algorithmiques d'Ouali

In [81]:
if ouali_to_validate_file and not ouali_align.empty:
    to_validate = pd.read_csv(ouali_to_validate_file, sep='\t', dtype = str)

    # Pour simplifier les manipulations, on copie les colonnes source et cible
    # On tente aussi d'en retirer certains caractères spéciaux qui pourraient fausser la comparaison.
    to_remove_pattern = re.compile(r'[\u2000-\u206F\u2E00-\u2E7F\u3000-\u303F]')
    to_validate['source'] = to_validate['id source'].str.replace(to_remove_pattern, '', regex=True).str.strip()
    to_validate['cible'] = to_validate['id cible'].str.replace(to_remove_pattern, '', regex=True).str.strip()
    to_validate['method'] = to_validate["décision d'alignement"]
    to_validate_align = to_validate.query("cible.notna() and not cible == '\"rnv-nz-auth-atc\"'").copy()
    to_validate_no_align = to_validate.query("cible.isna() and not cible == '\"rnv-nz-auth-atc\"' and not method == 'auto'").copy()
    print("Alignements à comparer aux alignements Ouali (à valider): ", len(to_validate_align))
    print("Non-alignements à comparer aux alignements Ouali (à valider): ", len(to_validate_no_align))

    outfile = output_folder + '/' + common_file + '-VALIDOUALI-' + set_name 
    common_source_validated = pd.merge(ouali_align, to_validate_align, how="inner", on='source', suffixes=("_ouali", "_avalider"), copy=True)
    common_target_validated = pd.merge(ouali_align, to_validate_align, how="inner", on=['source', 'cible'], suffixes=("_ouali", "_avalider"), copy=True)
    display(common_target_validated[['source','cible', 'forme principale source_ouali','forme principale cible_ouali','forme principale cible_avalider', "décision d'alignement_avalider", 'arbitre_avalider']])
    
    if output_type == 'xlsx':
        outfile = outfile + '.xlsx'
        common_target_validated.to_excel(outfile,columns=['source','cible', 'forme principale source_ouali','forme principale cible_ouali','forme principale cible_avalider', "décision d'alignement_avalider", 'arbitre_avalider'],index=False)
    else:
        outfile = outfile + '.csv'
        common_target_validated.to_csv(outfile, columns=['source','cible', 'forme principale source_ouali','forme principale cible_ouali','forme principale cible_avalider', "décision d'alignement_avalider", 'arbitre_avalider'], encoding="UTF-8",index=False)
    print("Alignements validés par les alignements Ouali: ", len(common_target_validated))
    print("Pourcentage validé: ", round(len(common_target_validated)/len(common_source_validated)*100,1), "%")
    print("Exportés dans ", outfile)

    # Alignements à valider pour lesquels Ouali a pris une décision différente (cible différente pour une même source)
    
    outfile = output_folder + '/' + divergences_file + '-VALIDOUALI-' + set_name
    divergences_to_validate = common_source_validated.loc[~(common_source_validated['cible_ouali'] == common_source_validated['cible_avalider'])]
    display(divergences_to_validate[['source', 'cible_ouali', 'cible_avalider', 'forme principale source_ouali', 'forme principale cible_ouali','forme principale cible_avalider', "décision d'alignement_avalider", 'arbitre_avalider']])
    
    if output_type == 'xlsx':
        outfile = outfile + '.xlsx'
        divergences_to_validate.to_excel(outfile,columns=['source', 'cible_ouali', 'cible_avalider', 'forme principale source_ouali', 'forme principale cible_ouali','forme principale cible_avalider', "décision d'alignement_avalider", 'arbitre_avalider'],index=False)
    else:
        outfile = outfile + '.csv'
        divergences_to_validate.to_csv(outfile, columns=['source', 'cible_ouali', 'cible_avalider', 'forme principale source_ouali', 'forme principale cible_ouali','forme principale cible_avalider', "décision d'alignement_avalider", 'arbitre_avalider'],encoding="UTF-8",index=False)
    print("Alignements qui divergent avec les alignements Ouali: ", len(divergences_to_validate))
    print("Pourcentage de divergence: ", round(len(divergences_to_validate)/len(common_source_validated)*100,1), "%")
    print("Exportés dans ", outfile)

    # Non-alignements pour lesquels Ouali a trouvé un alignement
    
    outfile = output_folder + '/' + div_noalign_file + '-VALIDOUALI-' + set_name
    div_nonalign_to_validate = pd.merge(ouali_align, to_validate_no_align, how="inner", on='source', suffixes=("_ouali", "_avalider"), copy=True)
    display(div_nonalign_to_validate[['source','cible_ouali', 'forme principale source_ouali', 'forme principale cible_ouali', "décision d'alignement_avalider", 'arbitre_avalider']])
    
    if output_type == 'xlsx':
        outfile = outfile + '.xlsx'
        div_nonalign_to_validate.to_excel(outfile,columns=['source','cible_ouali', 'forme principale source_ouali', 'forme principale cible_ouali', "décision d'alignement_avalider", 'arbitre_avalider'],index=False)
    else:
        outfile = outfile + '.csv'
        div_nonalign_to_validate.to_csv(outfile,columns=['source','cible_ouali', 'forme principale source_ouali', 'forme principale cible_ouali', "décision d'alignement_avalider", 'arbitre_avalider'],encoding="UTF-8",index=False)
    
    print("Non-alignements à valider pour lesquels Ouali à trouvé un alignement: ", len(div_nonalign_to_validate))
    print("Pourcentage de divergence: ", round(len(div_nonalign_to_validate)/len(to_validate_no_align)*100,1), "%")
    print("Exportés dans ", outfile)

Alignements à comparer aux alignements Ouali (à valider):  324150
Non-alignements à comparer aux alignements Ouali (à valider):  4665


,source,cible,forme principale source_ouali,forme principale cible_ouali,forme principale cible_avalider,décision d'alignement_avalider,arbitre_avalider
0,981023132829902851,119904187,"Ganjei-Azar, Parvin","Ganjei-Azar, Parvin","Ganjei-Azar, Parvin",externe,NaN
1,981023265030102851,193026813,"Luque-Ayala, Andrés","Luque-Ayala, Andrés","Luque-Ayala, Andrés",externe,NaN
2,981023265031702851,03367387X,"Moullec, Gaël-Georges","Moullec, Gaël 1962-....","Moullec, Gaël 1962-....",externe,NaN
3,981023265037102851,246629495,"Buisson, Nelly","Buisson, Nelly 19..-....","Buisson, Nelly 19..-....",externe,NaN
4,981023265132802851,260039209,"Martín Rodrigo, Inés, 1983-","Martín Rodrigo, Inés 1983-....","Martín Rodrigo, Inés 1983-....",externe,NaN
...,...,...,...,...,...,...,...
245862,981023879732102851,203060989,"Fury, Alexander","Fury, Alexander","Fury, Alexander",externe,NaN
245863,981023880027802851,243968450,"Emonot Moussadek, Marion","Emonot Moussadek, Marion","Emonot Moussadek, Marion",externe,NaN
245864,981023881122402851,193221381,"Crowther, John","Crowther, John C.","Crowther, John C.",externe,NaN
245865,981023899721702851,258013672,"Bhend, Julia","Bhend-Rutishauser, Julia 1975-....","Bhend-Rutishauser, Julia 1975-....",manuel,7450


Alignements validés par les alignements Ouali:  245867
Pourcentage validé:  98.9 %
Exportés dans  output/communs-VALIDOUALI-benchmark2024-min2align.xlsx


,source,cible_ouali,cible_avalider,forme principale source_ouali,forme principale cible_ouali,forme principale cible_avalider,décision d'alignement_avalider,arbitre_avalider
13,981023280150802851,102423873,029570239,"Dieter, 1958-","Teste, Didier",Dieter 1958-....,externe,NaN
77,981023280276202851,074645617,034589007,"Klein, Joe",Anonymus pseud.,"Klein, Joe 19..-....",externe,NaN
92,981023280304902851,087893657,172359066,"Rossier, Henri, 1835-1928","Rossier, Henri","Rossier, Henri 1835-1928 Collections d'art",externe,NaN
97,981023280313602851,140262342,147520681,"Beeby-Thompson, A","Beeby-Thompson, A.","Thompson, Arthur Beeby",externe,NaN
102,981023280337502851,056861605,066955858,"Thompson, Joyce Beebe","Thompson, Joyce E",NaN,externe,NaN
...,...,...,...,...,...,...,...,...
248268,981023764930902851,256796653,171218612,"Matisse, Jackie, 1931-2021","Matisse-Monnier, Jacqueline 1931-2021","Matisse, Jackie 1931-....",externe,NaN
248274,981023765026902851,265711894,092393896,"Raffarin, Anne","Raffarin, Anne","Raffarin, Anne 19..-....",externe,NaN
248288,981023765037602851,07711020X,05848986X,"Acquaroni, Rosana 1964-","Acquaroni, Rosana 1964-....","Acquaroni Muñoz, Rosana",externe,NaN
248594,981023878329802851,083907599,058504605,"Bresson-Hadni, Solange","Bresson, Solange 1958-....","Hadni-Bresson, Solange",externe,NaN


Alignements qui divergent avec les alignements Ouali:  2773
Pourcentage de divergence:  1.1 %
Exportés dans  output/divergences-VALIDOUALI-benchmark2024-min2align.xlsx


,source,cible_ouali,forme principale source_ouali,forme principale cible_ouali,décision d'alignement_avalider,arbitre_avalider
0,981023280174802851,125940912,"Reymond, Henry, 1863-1937","Reymond, Henri physicien",manuel,8153
1,981023281553202851,180919245,"Lapp, Patrick, 1944-","Lapp, Patrick 1944-....",externe,NaN
2,981023281753602851,269035842,"Roulin, Marc","Roulin, Marc 19..-....",manuel,7161
3,981023281764502851,255875649,"Rieder, Lukas","Rieder, Lukas 19..-....",externe,NaN
4,981023281850402851,256596085,"Tabin-Darbellay, Isabelle","Tabin-Darbellay, Isabelle‏ ‎ 1947-....",externe,NaN
...,...,...,...,...,...,...
326,981023906015802851,146239393,"Evans, Ray, 1915-2007","Evans, Ray",manuel,7450
327,981023982590202851,187863067,"Ferreira, Cristina, 1977-","Ferreira, Cristina",manuel,8560
328,981024004284802851,184437954,"Frossard, Marc","Frossard, Marc 1757-1815",manuel,7450
329,981024102181002851,244774196,"Wagner, Susanne, Professeur d'allemand","Wagner, Susanne 1968-....",manuel,8560


Non-alignements à valider pour lesquels Ouali à trouvé un alignement:  331
Pourcentage de divergence:  7.1 %
Exportés dans  output/divergences-nonalignements-VALIDOUALI-benchmark2024-min2align.xlsx


In [77]:
test = to_validate.query("source == '981023281692202851'")
test2 = ouali_align.query("source == '981023281692202851'")
display(test.cible.values[0])
display(test2.cible.values[0])

'257426078\u200f'

'257426078'

#### Validation d'alignements à l'aide des alignements communs Ouali et ABES
Si un fichier à valider a été fourni, celui-ci va maintenant être comparé aux alignements communs Ouali et ABES calculés
dans les étapes précédentes.

In [83]:
if ouali_to_validate_file and not common_target_abes.empty:
    to_validate = pd.read_csv(ouali_to_validate_file, sep='\t', dtype = str)
    
    # Pour simplifier les manipulations, on copie les colonnes source et cible
    # On tente aussi d'en retirer certains caractères spéciaux qui pourraient fausser la comparaison.
    to_remove_pattern = re.compile(r'[\u2000-\u206F\u2E00-\u2E7F\u3000-\u303F]')
    to_validate['source'] = to_validate['id source'].str.replace(to_remove_pattern, '', regex=True).str.strip()
    to_validate['cible'] = to_validate['id cible'].str.replace(to_remove_pattern, '', regex=True).str.strip()
    to_validate['method'] = to_validate["décision d'alignement"]
    to_validate_align = to_validate.query("cible.notna() and not cible == '\"rnv-nz-auth-atc\"'").copy()
    to_validate_no_align = to_validate.query("cible.isna() and not cible == '\"rnv-nz-auth-atc\"' and not method == 'auto'").copy()
    print("Alignements à comparer aux alignements communs (à valider): ", len(to_validate_align))
    print("Non-alignements à comparer aux alignements communs (à valider): ", len(to_validate_no_align))
    
    outfile = output_folder + '/' + common_file + '-CONSENSUS-' + set_name
    common_source_validated = pd.merge(common_target_abes, to_validate_align, how="inner", on='source', suffixes=("_oualivalid", "_avalider"), copy=True)
    common_target_validated = pd.merge(common_target_abes, to_validate_align, how="inner", on=['source', 'cible'], suffixes=("_oualivalid", "_avalider"), copy=True)
    display(common_target_validated[['source','cible', 'forme principale source', 'forme principale cible_oualivalid','forme principale cible_avalider', "décision d'alignement", 'arbitre']])
    
    if output_type == 'xlsx':
        outfile = outfile + '.xlsx'
        common_target_validated.to_excel(outfile,columns=['source','cible', 'forme principale source', 'forme principale cible_oualivalid','forme principale cible_avalider', "décision d'alignement", 'arbitre'],index=False)
    else:
        outfile = outfile + '.csv'
        common_target_validated.to_csv(outfile,columns=['source','cible', 'forme principale source', 'forme principale cible_oualivalid','forme principale cible_avalider', "décision d'alignement", 'arbitre'],encoding="UTF-8",index=False)
    print("Alignements validés par les alignements communs: ", len(common_target_validated))
    print("Pourcentage validé: ", round(len(common_target_validated)/len(common_source_validated)*100,1), "%")
    print("Exportés dans ", outfile)

    # Alignements à valider pour lesquels Ouali et l'ABES ont pris une décision différente (cible différente pour une même source)
    
    outfile = output_folder + '/' + divergences_file + '-CONSENSUS-' + set_name
    divergences_to_validate = common_source_validated.loc[~(common_source_validated['cible_oualivalid'] == common_source_validated['cible_avalider'])]
    display(divergences_to_validate[['source', 'cible_oualivalid', 'cible_avalider', 'forme principale source', 'forme principale cible_oualivalid','forme principale cible_avalider', "décision d'alignement", 'arbitre']])

    if output_type == 'xlsx':
        outfile = outfile + '.xlsx'
        divergences_to_validate.to_excel(outfile, columns=['source', 'cible_oualivalid', 'cible_avalider', 'forme principale source', 'forme principale cible_oualivalid','forme principale cible_avalider', "décision d'alignement", 'arbitre'],index=False)
    else:
        outfile = outfile + '.csv'
        divergences_to_validate.to_csv(outfile, columns=['source', 'cible_oualivalid', 'cible_avalider', 'forme principale source', 'forme principale cible_oualivalid','forme principale cible_avalider', "décision d'alignement", 'arbitre'],encoding="UTF-8",index=False)
    print("Alignements qui divergent avec les alignements communs: ", len(divergences_to_validate))
    print("Exportés dans ", outfile)

    # Non-alignements pour lesquels Ouali et l'ABES ont trouvé un alignement
    
    outfile = output_folder + '/' + div_noalign_file + '-CONSENSUS-' + set_name
    div_nonalign_to_validate = pd.merge(common_target_abes, to_validate_no_align, how="inner", on='source', suffixes=("_oualivalid", "_avalider"), copy=True)
    display(div_nonalign_to_validate[['source','cible_oualivalid', 'forme principale source', 'forme principale cible_oualivalid', "décision d'alignement", 'arbitre']])
    
    if output_type == 'xlsx':
        outfile = outfile + '.xlsx'
        div_nonalign_to_validate.to_excel(outfile,columns=['source','cible_oualivalid', 'forme principale source', 'forme principale cible_oualivalid', "décision d'alignement", 'arbitre'],index=False)
    else:
        outfile = outfile + '.csv'
        div_nonalign_to_validate.to_csv(outfile,columns=['source','cible_oualivalid', 'forme principale source', 'forme principale cible_oualivalid', "décision d'alignement", 'arbitre'],encoding="UTF-8",index=False)
    
    print("Non-alignements Ouali erronés: ", len(div_nonalign_to_validate))
    print("Pourcentage de divergence: ", round(len(div_nonalign_to_validate)/len(to_validate_no_align)*100,1), "%")
    
    print("Exportés dans ", outfile)


Alignements à comparer aux alignements communs (à valider):  324150
Non-alignements à comparer aux alignements communs (à valider):  4665


,source,cible,forme principale source,forme principale cible_oualivalid,forme principale cible_avalider,décision d'alignement,arbitre
0,981023132829902851,119904187,"Ganjei-Azar, Parvin","Ganjei-Azar, Parvin","Ganjei-Azar, Parvin",externe,NaN
1,981023265030102851,193026813,"Luque-Ayala, Andrés","Luque-Ayala, Andrés","Luque-Ayala, Andrés",externe,NaN
2,981023265031702851,03367387X,"Moullec, Gaël-Georges","Moullec, Gaël 1962-....","Moullec, Gaël 1962-....",externe,NaN
3,981023265037102851,246629495,"Buisson, Nelly","Buisson, Nelly 19..-....","Buisson, Nelly 19..-....",externe,NaN
4,981023265132802851,260039209,"Martín Rodrigo, Inés, 1983-","Martín Rodrigo, Inés 1983-....","Martín Rodrigo, Inés 1983-....",externe,NaN
...,...,...,...,...,...,...,...
211098,981023303639002851,03411839X,Morrissey,"Morrissey, Steven Patrick 1959-.......","Morrissey, Steven Patrick 1959-.......",externe,NaN
211099,981023303818402851,254056431,"Pociao, 1951-",Pociao 1951-....,Pociao 1951-....,externe,NaN
211100,981023364077402851,031776124,"Stan, 1969-",Stan 1969-....,Stan 1969-....,manuel,7167
211101,981023384829602851,156495597,Krystel,Krystel 1985-....,Krystel 1985-....,manuel,8542


Alignements validés par les alignements communs:  211103
Pourcentage validé:  100.0 %
Exportés dans  output/communs-CONSENSUS-benchmark2024-min2align.xlsx


,source,cible_oualivalid,cible_avalider,forme principale source,forme principale cible_oualivalid,forme principale cible_avalider,décision d'alignement,arbitre
50,981023281580002851,095766871,121435369,"Kloevekorn, Fritz, 1885-1964","Kloevekorn, Fritz","Klövekorn, Fritz 1885-19..",manuel,8542
76,981023282166302851,035347104,067282113,"Keller, Max, Dr en droit","Keller, Max 1945-....","Keller, Max Prof. Dr. jur. 1924-2003",manuel,7383
191,981023285670002851,034445161,273212192,"Piguet, Olivier, musicien","Piguet, Olivier 1942-....","Piguet, Olivier chef de choeur 1963-....",manuel,7450
447,981023295461302851,081223196,079384293,"Schmidt-Boelcke, Werner, 1903-1985","Schmidt-Boelcke, Werner","Schmidt-Boelcke, Werner 1903-1985",manuel,7450
14197,981023309750502851,195924231,275654079,"Bosshardt, Walter","Bosshardt, Walter 1890-19..","Bosshardt, Walter graphiste 19..-....",externe,NaN
102208,981023345066802851,03509141X,277085446,"Schoch, Rudolf","Schoch, Rudolf germaniste 1850-1920","Schoch, Rudolf musicologue 1896-1982",externe,NaN
120094,981023352150002851,256928169,A010215007,"Weissen, Andreas","Weissen, Andreas",NaN,externe,NaN
123014,981023353308902851,030160782,270930884,"Huber, Rudolf, Dr en sc. éc","Huber, Rudolf 1913-....","Huber, Rudolf économiste 1955-....",manuel,7383
210743,981023282853002851,059441356,067181597,La Mothe,La Mothe ....-1740?,NaN,externe,NaN
210802,981023294433802851,032403720,060949325,"Henri, de Valenciennes",Henri de Valenciennes 1170?-12..,"Valenciennes, Pierre Henri de 1750-1819",externe,NaN


Alignements qui divergent avec les alignements communs:  15
Exportés dans  output/divergences-CONSENSUS-benchmark2024-min2align.xlsx


,source,cible_oualivalid,forme principale source,forme principale cible_oualivalid,décision d'alignement,arbitre
0,981023282035502851,254652689,"Schuler, Peter","Schuler, Peter",manuel,8086
1,981023284601202851,060649119,"Mehta, Ramanlal Nagarji","Mehta, Ramanlal Nagarji 1922-....",manuel,8542
2,981023303625302851,077188799,"Dubuc, Guillaume","Dubuc, Guillaume pharmacien rouennais 1764-1837",manuel,8086
3,981023328019702851,257012796,"Kaboth, Berta","Kaboth, Berta",manuel,8542
4,981023285198202851,262201070,Anouk,Anouk 1953-....,manuel,7461
5,981023296450802851,052575470,"Nārāyaṇa, protégé de Dhavalacandra",Nārāyaṇa 1426-14..,externe,NaN


Non-alignements Ouali erronés:  6
Pourcentage de divergence:  0.1 %
Exportés dans  output/divergences-nonalignements-CONSENSUS-benchmark2024-min2align.xlsx
